# Practicum: Modern Hopfield Network Spell Checking and Word Recommendation
___
In this practicum problem, we'll implement a modern Hopfield Network and use it for spell checking and word recommendation tasks.

* _What are Hopfield Networks_? Hopfield Networks are used for associative memory, where the network can recall a pattern from a partial input. The modern version of Hopfield Networks uses continuous values instead of binary values, and it can store multiple patterns. 
* We'll use the following paper to guide our implementation and analysis: [Ramsauer, H., Schafl, B., Lehner, J., Seidl, P., Widrich, M., Gruber, L., Holzleitner, M., Pavlovi'c, M., Sandve, G.K., Greiff, V., Kreil, D.P., Kopp, M., Klambauer, G., Brandstetter, J., & Hochreiter, S. (2020). Hopfield Networks is All You Need. ArXiv, abs/2008.02217.](https://arxiv.org/abs/2008.02217)

## Tasks
Before we get started, we'll quickly review modern Hopfied Networks. Then, you'll execute the `Run All Cells` command to check if you (or your neighbor) have any code or setup issues. Code issues, then raise your hands - and let's get those fixed!

* __Task 1: Setup, Data, Constants (5 min)__: Let's take 5 minutes to load [a Simpsons character library from Kaggle](https://www.kaggle.com/datasets/kostastokis/simpsons-faces) that our Hopfield network will memorize.
*  __Task 2: Build a Modern Network Model (5 min)__: In this task, we'll formulate the image dataset we give the network and then create a model of a modern Hopfield network. We'll also quickly check to ensure we are doing what we think we are doing.
* __Task 3: Retrieve a memory from the network (30 min)__: In this task, we will retrieve a memory from the modern Hopfield network starting from a random state vector $\mathbf{s}_{\circ}$. We'll corrupt an image (by cutting off some fraction of the image) and then see if the model recovers the correct memory given the corrupted starting point. 

Let's get started!

___

## Background
A modern Hopfield network addresses many of the perceived limitations of the original Hopfield network. The original Hopfield network was limited to binary values and could only store a limited number of patterns. The modern Hopfield network uses continuous values and can store a large number of patterns.
* For a detailed discussion of the key milestones in the development of modern Hopfield networks, check out [Hopfield Networks is All You Need Blog, GitHub.io](https://ml-jku.github.io/hopfield-layers/)

### Algorithm
The user provides a set of memory vectors $\mathbf{X} = \left\{\mathbf{x}_{1}, \mathbf{x}_{2}, \ldots, \mathbf{x}_{m}\right\}$, where $\mathbf{x}_{i} \in \mathbb{R}^{n}$ is a memory vector of size $n$ and $m$ is the number of memory vectors. Further, the user provides an initial _partial memory_ $\mathbf{s}_{\circ} \in \mathbb{R}^{n}$, which is a vector of size $n$ that is a partial version of one of the memory vectors and specifies the _temperature_ $\beta$ of the system.

__Initialize__ the network with the memory vectors $\mathbf{X}$, and the inverse temperature $\beta$. Set current state to the initial state $\mathbf{s} \gets \mathbf{s}_{\circ}$

Until convergence __do__:
   1. Compute the _current_ probability vector defined as $\mathbf{p} = \texttt{softmax}(\beta\cdot\mathbf{X}^{\top}\mathbf{s})$ where $\mathbf{s}$ is the _current_ state vector, and $\mathbf{X}^{\top}$ is the transpose of the memory matrix $\mathbf{X}$.
   2. Compute the _next_ state vector $\mathbf{s}^{\prime} = \mathbf{X}\mathbf{p}$ and the _next_ probability vector $\mathbf{p}^{\prime} = \texttt{softmax}(\beta\cdot\mathbf{X}^{\top}\mathbf{s}^{\prime})$.
   3. If $\mathbf{p}^{\prime}$ is _close_ to $\mathbf{p}$ or we run out of iterations, then __stop__. For example, $\lVert \mathbf{p}^{\prime} - \mathbf{p}\rVert_{2}^{2} \leq \epsilon$ for some small $\epsilon > 0$.
   4. Otherwise, update the state $\mathbf{s} \gets\mathbf{s}^{\prime}$, and __go back to__ step 1.

   
This algorithm is implemented in [the `recover(...)` method](src/Compute.jl).

## Task 1: Setup, Data, and Prerequisites
We set up the computational environment by including the `Include.jl` file, loading any needed resources, such as sample datasets, and setting up any required constants. 
* The `Include.jl` file also loads external packages, various functions that we will use in the exercise, and custom types to model the components of our problem. It checks for a `Manifest.toml` file; if it finds one, packages are loaded. Other packages are downloaded and then loaded.

In [1]:
include("Include.jl"); # load a bunch of libs, including the ones we need to work with images

### Constants
Before we load the data, let's set up some constants that we will use in the exercise. 

__TODO__: Please enter the number of words (memories) you want your Hopefield network to memorize, the embedding dimension and the inverse temperature $\beta$.

* The `number_of_words_to_memorize::Int` should be less than or equal to the number of words in the dataset that we load below.
* The `number_of_embedding_dimesions::Int` depends upon the pretrained embedding dataset that we will use. In this case, we will use the [GloVe](https://nlp.stanford.edu/projects/glove/) dataset, with `50` embedding dimensions.
* The inverse temperature $\beta > 0$ is a hyperparameter that controls the sharpness of the softmax distribution. A higher value of $\beta$ will make the distribution sharper, while a lower value will make it smoother. Set an initial value of `0.5` for $\beta$.

In [2]:
number_of_words_to_memorize = 2^5; # TODO: Enter how many mempories we want to memorize (Int ≥ 0)
number_of_embedding_dimesions = 50; # TODO: Enter the number of embedding dimensions for each word (Int = 50)
β = 1.5; # TODO: Enter an inverse temperature of the system (high T -> low β) Float64

### Data
In this section, let's load a vector embedding from words. We'll use the [GloVe pretrained word embedding dataset](https://nlp.stanford.edu/projects/glove/) to as the memories for our Hopfield model. The **GloVe (Global Vectors for Word Representation)** dataset is a widely used pre-trained word embedding resource developed by Pennington, Socher, and Manning at Stanford University. 
* _What is it?_ It constructs vector representations of words by aggregating global word co-occurrence statistics from a corpus, enabling semantic relationships to be captured in vector space. GloVe embeddings have been trained on large datasets such as Wikipedia, Gigaword, and Common Crawl, offering dimensionalities typically ranging from 50 to 300. These embeddings are foundational for many NLP tasks, including text classification, sentiment analysis, and machine translation.
* See: [Pennington et al., EMNLP 2014, GloVe: Global Vectors for Word Representation](https://aclanthology.org/D14-1162/) for more details on this dataset.

This dataset is large, so we won't check it into the repository. Instead, we'll download it from the internet. Fill me in.

In [3]:
word2vec, vec2word = let

    # do we have the embeddings downloaded?
    data = nothing;
    if (isfile(joinpath(_PATH_TO_DATA, "glove_6B_50d.jld2")) == false) 
        # TODO: download logic goes here ...
    else
        data = JLD2.load(joinpath(_PATH_TO_DATA, "glove_6B_50d.jld2")) # Ok, we have the embeddings file, so let's load it
    end

    # load the embeddings
    word2vec = data["word2vec"] # this is a Dict{String, Tuple{Float32}}
    vec2word = data["vec2word"] # this is a Dict{Tuple{Float32}, String}

    # return -
    (word2vec, vec2word)
end;

__Test data__: Now that we have loaded the pretrained GloVe dataset, let's select a (random) subset of length `number_of_words_to_memorize` from the dataset to encode into the modern Hopfield network. 
* This code block returns the `test_words::Array{String,1}` array of words that we will use to test the Hopfield network. The embedding vectors corresponding to these words are stored in the `test_vocabulary::Array{Float64,2}` array, where the vector representation of each word is stored in the columns of this array.

In [58]:
test_words, test_vocabulary = let

    # initialize -
    vocabulary = Array{Float64,2}(undef, number_of_words_to_memorize, number_of_embedding_dimesions); # this is a matrix of Float64
    test_words = Array{String,1}(undef, number_of_words_to_memorize); # this is a vector of strings
    total_number_of_words = length(word2vec); # this is the total number of words in the dataset
    index_of_words_to_learn = randperm(total_number_of_words)[1:number_of_words_to_memorize]; # this is the random index of words to learn

    # get the keys of word2vec -
    words = keys(word2vec) |> collect; # this is a vector of strings

    # loop over the words to learn
    for i ∈ eachindex(index_of_words_to_learn)
        
        j = index_of_words_to_learn[i]; # this is the index of the word we want to learn
        wⱼ = words[j]; # this is the word we want to learn
        test_words[i] = wⱼ; # this is the word we want to learn 
        embedding = word2vec[wⱼ]; # this is the embedding of the word we want to learn

        for j ∈ 1:number_of_embedding_dimesions
            vocabulary[i,j] = embedding[j]; # this is the embedding of the word we want to learn
        end
    end

    test_vocabulary = vocabulary |> transpose |> Matrix; # this is the vocabulary we want to learn

    # return -
    (test_words, test_vocabulary);
end;

In [59]:
test_words

32-element Vector{String}:
 "mckone"
 "matraville"
 "signallers"
 "scei"
 "belisle"
 "subservient"
 "infallible"
 "agaricales"
 "hillmon"
 "coptic"
 ⋮
 "sagd"
 "50.17"
 "crimen"
 "videanu"
 "frisco"
 "chot"
 "neufville"
 "jains"
 "maldacena"

__Check__: Let's check that the `test_words` and `test_vocabulary` arrays are of the correct size. 
* The `test_words` array should be of size `number_of_words_to_memorize`. The `test_vocabulary` array should be of size `number_of_embedding_dimensions` $\times$ `number_of_words_to_memorize`, i.e., the memorized words should be on the columns of the `test_vocabulary` array.
* We'll use the [@assert macro](hhttps://docs.julialang.org/en/v1/base/base/#Base.@assert) to check that the arrays are of the correct size. If the assertion fails, an error will be raised and the program will stop. If no error is raised, the program will continue (everything is correctly sized).

In [60]:
let
    @assert length(test_words) == number_of_words_to_memorize;
    @assert size(test_vocabulary, 1) == number_of_embedding_dimesions;
    @assert size(test_vocabulary, 2) == number_of_words_to_memorize;
end

## Task 2: Can we recover an uncorrupted memory?
In this task, we'll create a modern Hopfield network model, load the test data, and then check if we can recover an _uncoruppted_ memory from the network. We'll do this by starting from a state vector $\mathbf{s}_{\circ}$ that is a column from the data loaded into the model.

* __Convergence__: We are _guaranteed_ that the network will converge to a local minimum, but we are not guaranteed that the local minimum corresponds to the the original _uncoruppted_ memory.
* __Expectation__: We expect to recover the original _uncoruppted_ memory with a small number of mistakes when the system temperature is cold, i.e., $\beta > \beta^{\star}$, where $\beta^{\star}$ is a (unknown) threshold temperature that (potentially) depends on the number of memories we are memorizing. We'll play around with the inverse temperature below.
 

Let's start by creating a model of a modern Hopfield network. 
* We'll construct [a `MyModernHopfieldNetworkModel` instance](src/Types.jl) using a custom [`build(...)` function](src/Factory.jl). The [`build(...)` method](src/Factory.jl) takes the type of thing we want to build, the (linearized) image library we want to encode, and the (inverse) system temperature $\beta$ as inputs — images along the columns.
* The [`build(...)` function](src/Factory.jl) returns a `MyModernHopfieldNetworkModel` instance, where the image library is stored in the `X::Array{Float64,2}` field, and the system temperature is stored in the `β::Float64` field.

We'll store the Hopfield network instance in the `mymodel::MyModernHopfieldNetworkModel` variable.

In [61]:
mymodel = let

    # initialize -
    model = nothing; # this is the model we want to build
    memorycollection =test_vocabulary; # words (memories) on columns
    index_vector = 1:number_of_words_to_memorize |> collect; # this is the index of the words we want to learn
    words = keys(test_vocabulary) |> collect; # this is a vector of strings
    
    # build model -
    model = build(MyModernHopfieldNetworkModel, (
            memories = memorycollection, # this is the data we want to memorize. Images on columns
            β = β, # Inverse temperature of the system. A big beta means we are more likely to get the right answer
    ));

    model; # return the model to the calling scope
end;

__Check__: Let's do a quick check to make sure we are doing what we think we are doing when we loaded the memories into the model. The columns of the `model.X` field should be the words that we are encoding into the Hopfield network. Thus, we should be able to grab a column from `model.X` and look it up in the original `vec2word` dictionary.
* We'll use the [@assert macro](hhttps://docs.julialang.org/en/v1/base/base/#Base.@assert) to check that the true word, and the word encoded in the model are the same. If the assertion fails, an error will be raised and the program will stop. If no error is raised, the program will continue (everything is correct).

In [62]:
let
    
    # initialize -
    X = mymodel.X; # get the training data in the model
    index_to_check = rand(1:number_of_words_to_memorize); # what index do we want to check? (random)
    
    # Get the true word, and the word we think we learned -
    eᵢ = X[:,index_to_check] |> Tuple # this is the embedding of the word we want to learn
    wᵢ = test_words[index_to_check]; # this is the word we want to learn
    ŵᵢ = vec2word[eᵢ]; # this is the word we think we learned

    # Compare the two words -
    @assert wᵢ == ŵᵢ; # this is the word we want to learn
end

__Retrieve a memory from the network__: Next, we'll test if we can recover uncorrupted and corrupted memories from the Hopfield network.
Let's start by specifying which memory we are trying to recover in the `memoryindextorecover::Int` variable.

In [63]:
memoryindextorecover = 21; # TODO: Specify which memory vector will we choose (must be between 1 and number_of_words_to_memorize)

Next, let's build an uncorrupted and corrupted initial condition vector using the true word emebedding vector. We'll store 
the uncorrupted word in the `sₒ::Array{Float64,1}` variable, while the corrupted word will be stored in the `s₁::Array{Float64,1}` variable. Let's start with the uncorrupted memory.

In [64]:
sₒ = mymodel.X[:,memoryindextorecover]; # this is the memory vector we want to recover

What word does `memoryindextorecover::Int64` point to? Fill me in.

In [65]:
let 
    X = mymodel.X; # get the training data in the model
    p = β*(transpose(X) * sₒ) |> s-> NNlib.softmax(s) # this is the probability of the word we want to learn
    ŵ = argmax(p) |> i-> test_words[i]; # this is the index of the word we think we learned
    w = test_words[memoryindextorecover]; # this is the word we want to learn
    println("The word at index $(memoryindextorecover) that we (think) we encoded is: $(ŵ). Check: the true word is: ", w);
end

The word at index 21 that we (think) we encoded is: verkin. Check: the true word is: verkin


Now that we have a starting memory encoded in the state vector $\mathbf{s}_{\circ}$, can we recover the original uncorrupted word? We are guaranteed a word, but maybe _not_ the correct one.
* _Implementation_: We implemented the modern Hopfield recovery algorithm above in [the `recover(...)` method](src/Compute.jl). This method takes our `model::MyModernHopfieldNetworkModel` instance, the initial configuration vector `sₒ::Array{Float64,1}`, the maximum number `maxiterations::Int64`, and an iteration tolerance parameter `ϵ::Float64`. This method will continue to iterate until the probability vector converges, or we run out of iterations.
* [The `recover(...)` method](src/Compute.jl) returns the recovered word vector in the `ŝₒ::Array{Float32,1}` variable, the word vectors at each iteration are stored in the `fₒ::Dict{Int, Array{Float64,2}}` dictionary, and the probability of the words at each iteration in the `pₒ::Dict{Int, Array{Float64,2}}` variable. The dictionaries are indexed from `0`.

In [66]:
(ŝₒ,fₒ,pₒ) = recover(mymodel, sₒ, maxiterations = 10000, ϵ = 1e-16); # iterate until we hit stop condition

How many iterations did it take to converge? (this will be the length, i.e.,. the numbr of keys of the `fₒ` dictionary).

In [67]:
println("How many iterations: $(length(fₒ))") # how many iterations did we need to converge?

How many iterations: 3


__Which word did we recover?__ We can check if the recovered word is what we expected by looking at the probability of the recovered words stored in the `pₒ::Dict{Int, Array{Float64,2}}` dictionary.

In [68]:
recovered_word_uncorrupted = let 
    
    # initialize -
    number_of_iterations = length(fₒ); # how many iterations did we need to converge? 
    p = pₒ[number_of_iterations - 1]; # this is the probability of the word we want to learn  (0 based)
    ŵ = argmax(p) |> i-> test_words[i]; # this is the index of the word we think we learned
    
    ŵ; # return the word we *think* we learned
end

"verkin"

__Check__: Let's check to see if the recovered word is identical to the original word (not guaranteed). We can do this by checking the `s₁::Array{Float32,1}` variable against the original image.

In [69]:
let
    true_word = test_words[memoryindextorecover]; # this is the word we want to learn
    @assert recovered_word_uncorrupted == true_word
end

## Task 3: Retrieve a corrupted memory from the network
In this task, we'll repeat the process above, but this time we'll start from a corrupted memory. We'll do this by cutting off a fraction of the word and then see if the model recovers the correct memory given the corrupted starting point. 

Let's get started by building a corrupted memory. We'll iterate through each embedding dimension from the uncorrupted word; sometimes, we'll make a mistake and replace the correct embedding value with an incorrect value. We'll control how oftern we make a mistake using the hyperparameter $\theta$.
* _What is the $\theta$ parameter?_ The $\theta$ hyperparameter controls how often we make mistakes. Its interpretation depends upon our _mistake_ model. For example, if we are cutting off some fraction of the embedding dimension, then $\theta$ describes the fraction of the image we are cutting off. 

Whichever mistake model we use, the $\theta\in[0,1]$. We store the corrupted word in the `s₁::Array{Float64,1}` variable.

In [70]:
s₁ = let

    # initialize -
    sₒ = mymodel.X[:,memoryindextorecover]; # this is the memory vector we want to recover (uncorrupted)
    s₁ = Array{Float32,1}(undef, number_of_embedding_dimesions); # initialize some space to store the corrupted word
    θ = 0.4; # TODO: set a mistake threshold (1 - θ is the fraction the original memory that we retain)

    # Corruption model: Cutoff part of the memory
    cutoff = (1-θ)*number_of_embedding_dimesions |> x-> round(Int,x);
    for i ∈ 1:number_of_embedding_dimesions
        eᵢ =  sₒ[i]; # We have some gray-scale values in the original vector, need to perturb
        if (i ≤ cutoff)
            s₁[i] = eᵢ;
        else
            s₁[i] = β*randn(); # add some random noise (proportional to β)
        end
    end
    
    s₁ # return corrupted data to the calling scope
end;

__What is the closet word to the corrupted word__? Using the inner product self attention mechanism as our measure of similarity, we can compute the most probable memory given the corrupted memory. 

In [71]:
let
    # initialize -
    number_of_top_words = 5; # TODO: You can see how many words are the closest to the corrupted memory by changing this number
    X = mymodel.X; # get the training data in the model
    p = β*(transpose(X) * s₁) |> s-> NNlib.softmax(s) # this is the probability of the word we want to learn
    ŵ = argmax(p) |> i-> test_words[i]; # this is the index of the word we think we learned

    # make a table -
    df = DataFrame();
    sorted_indices = sortperm(p, rev=true); # sort the indices of the probabilities
    for i ∈ 1:number_of_top_words
        index = sorted_indices[i]; # this is the index of the word we think we learned
        ŵ = test_words[index]; # this is the word we think we learned
        p̂ = p[index]; # this is the probability of the word we think we learned
        push!(df, (word=ŵ, index = index, probability=p̂)); # add the word and its probability to the table
    end
    pretty_table(df, tf = tf_simple)
end

============= ======= ==============
        word   index   probability 
      String   Int64       Float64 
============= ======= ==============
      verkin      21      0.998894
  agaricales       8   0.000455889
  infallible       7   0.000308335
      crimen      26    0.00020042
   neufville      30    9.22495e-5
============= ======= ==============


__Do we converge to the correct word starting from corrupted word__? Now that we have a starting (corrupted) memory encoded in the $\mathbf{s}_{1}$ state vector, can we recover the original uncorrupted memory, i.e., the uncorrputed word? We are guaranteed to converge to a word, but maybe _not_ the correct one.
* _Implementation_: We implemented the modern Hopfield recovery algorithm above in [the `recover(...)` method](src/Compute.jl). This method takes our `model::MyModernHopfieldNetworkModel` instance, the initial configuration vector `sₒ::Array{Int32,1}`, and the maximum number `maxiterations::Int64`, and iteration tolerance parameter `ϵ::Float64`. 
* [The `recover(...)` method](src/Compute.jl) returns the recovered image in the e`s₁::Array{Float32,1}` variable, the image at each iteration in the `f::Dict{Int, Array{Float32,2}}` dictionary, and the probability of the image at each iteration in the `p::Dict{Int, Array{Float32,2}}` variable. The frames and probability dictionaries are indexed from `0`.

In [72]:
(ŝ₁,f₁,p₁) = recover(mymodel, s₁, maxiterations = 10000, ϵ = 1e-16); # iterate until we hit stop condition

How many iterations did it take to converge? 

In [73]:
println("The network converged to an answer (staring from a corrupted word) in: $(length(f₁)) iterations.") # how many iterations did we need to converge?

The network converged to an answer (staring from a corrupted word) in: 5 iterations.


In [74]:
recovered_word_corrupted = let 
    
    # initialize -
    number_of_iterations = length(f₁); # how many iterations did we need to converge? 
    p = p₁[number_of_iterations - 1]; # this is the probability of the word we want to learn  (0 based)
    ŵ = argmax(p) |> i-> test_words[i]; # this is the index of the word we think we learned
    
    ŵ; # return the word we *think* we recovered
end

"verkin"

## Tests
In the code block below, we check some values in your notebook and give you feedback on which items are correct or different. `Unhide` the code block below (if you are curious) about how we implemented the tests and what we are testing.

In [75]:
let 
    @testset verbose = true "CHEME 5820 Practicum S2025" begin

        @testset "Task 1: Setup, Prerequisites and Data" begin
            @test _DID_INCLUDE_FILE_GET_CALLED == true
            @test isnothing(number_of_words_to_memorize) == false
            @test isnothing(number_of_embedding_dimesions) == false
            @test isnothing(β) == false
            @test isnothing(word2vec) == false
            @test length(test_words) == number_of_words_to_memorize;
            @test size(test_vocabulary, 1) == number_of_embedding_dimesions;
            @test size(test_vocabulary, 2) == number_of_words_to_memorize;
        end

        @testset "Task 2: Recovering a word from a memory vector" begin
            @test isnothing(mymodel) == false
            @test size(mymodel.X, 1) == number_of_embedding_dimesions
            @test size(mymodel.X, 2) == number_of_words_to_memorize
            @test isnothing(sₒ) == false
            @test length(sₒ) == number_of_embedding_dimesions
            @test isnothing(memoryindextorecover) == false
            @test length(fₒ) > 0
            @test length(pₒ) > 0
        end

        @testset "Task 3: Recovering a word from a corrupted memory vector" begin
            @test isnothing(s₁) == false
            @test length(s₁) == number_of_embedding_dimesions
            @test length(f₁) > 0
            @test length(p₁) > 0
        end
    end
end

Test Summary:                                              | Pass  Total  Time
CHEME 5820 Practicum S2025                                 |   20     20  0.0s
  Task 1: Setup, Prerequisites and Data                    |    8      8  0.0s
  Task 2: Recovering a word from a memory vector           |    8      8  0.0s
  Task 3: Recovering a word from a corrupted memory vector |    4      4  0.0s


Test.DefaultTestSet("CHEME 5820 Practicum S2025", Any[Test.DefaultTestSet("Task 1: Setup, Prerequisites and Data", Any[], 8, false, false, true, 1.746362929225082e9, 1.746362929225126e9, false, "/Users/jeffreyvarner/Desktop/julia_work/CHEME-5820-SP25/CHEME-5820-Practicum-S2025/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X65sZmlsZQ==.jl"), Test.DefaultTestSet("Task 2: Recovering a word from a memory vector", Any[], 8, false, false, true, 1.746362929225135e9, 1.746362929225167e9, false, "/Users/jeffreyvarner/Desktop/julia_work/CHEME-5820-SP25/CHEME-5820-Practicum-S2025/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X65sZmlsZQ==.jl"), Test.DefaultTestSet("Task 3: Recovering a word from a corrupted memory vector", Any[], 4, false, false, true, 1.746362929225174e9, 1.746362929225189e9, false, "/Users/jeffreyvarner/Desktop/julia_work/CHEME-5820-SP25/CHEME-5820-Practicum-S2025/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X65sZmlsZQ==.jl")], 0, false, true, true, 1.74636292922505